In [24]:
import mpilot
import arcpy
import inspect
import pandas as pd
import geopandas as gpd
import os
import shutil
from shapely import wkt
from pyproj import CRS

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

In [25]:
df = pd.read_csv("out_data/eems/A_RU_8.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18180 entries, 0 to 18179
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Join_ID     18180 non-null  int64  
 1   apn         18179 non-null  object 
 2   Pr_Lg_Fr    18180 non-null  float64
 3   Pr_Fi_Ign   18180 non-null  float64
 4   Pc_Cg_LgFi  18180 non-null  float64
 5   Pc_Fi_Ign   18180 non-null  float64
 6   Mj_LU_Cat   18180 non-null  int64  
 7   Shape_Leng  18180 non-null  float64
 8   Shape_Area  18180 non-null  float64
 9   Fl_Fa       18180 non-null  float64
 10  WUI_Interm  18180 non-null  int64  
 11  WUI_Interf  18180 non-null  int64  
 12  geometry    18180 non-null  object 
dtypes: float64(7), int64(4), object(2)
memory usage: 1.8+ MB


In [32]:
def get_base_str(reporting_units_path, output_path):
    base_str = f"""

# Read Commands
Join_ID = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Join_ID",
    DataType = "Integer",
    Metadata = [
        DisplayName: Join&nbsp;ID,
        Description: Join&nbsp;ID,
        ColorMap: RdYlBu_r
    ]
)

Mj_LU_Cat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Mj_LU_Cat",
    DataType = "Integer",
    Metadata = [
        DisplayName: Majority&nbsp;LandUse&nbsp;Category&nbsp;Raw,
        Description: Majority&nbsp;LandUse&nbsp;Category&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pr_Lg_Fr = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Lg_Fr",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire&nbsp;Raw,
        Description: Relative&nbsp;Probability&nbsp;Of&nbsp;Large&nbsp;Fire&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pr_Fi_Ign = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Fi_Ign",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pc_Cg_LgFi = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pc_Cg_LgFi",
    DataType = "Float",
    Metadata = [
        DisplayName: Average&nbsp;Percent&nbsp;Change&nbsp;In&nbsp;Large&nbsp;Fire&nbsp;Probability&nbsp;Raw,
        Description: Avg&nbsp;Percent&nbsp;Change&nbsp;In&nbsp;Lg&nbsp;Fire&nbsp;Prob&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Fl_Fa = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Fl_Fa",
    DataType = "Float",
    Metadata = [
        DisplayName: Flora&nbsp;And&nbsp;Fauna&nbsp;Raw,
        Description: Flora&nbsp;And&nbsp;Fauna&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interm = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interm",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Intermix,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interf = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interf",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Interface,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Fz = CvtToFuzzyCat(
    InFieldName = Mj_LU_Cat, 
    RawValues = [0,1,2,3,4], 
    FuzzyValues = [1,0,0.1,0,0], 
    DefaultFuzzyValue = 0,
    Metadata = [
        DisplayName: Fuzzy&nbsp;LandUse&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Cvt = CvtFromFuzzy(
    InFieldName = LU_Cat_Fz, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: LandUse&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

# Normalizations

Nm_Fl_Fa = Normalize(
    InFieldName = Fl_Fa, 
    StartVal = 1, 
    EndVal = 0,
    Metadata = [
        DisplayName: Low&nbsp;Ecological&nbsp;Importance,
        ColorMap: RdYlBu_r
    ]
)


Nm_Lg_Fire = Normalize(
    InFieldName = Pr_Lg_Fr, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire,
        ColorMap: RdYlBu_r
    ]
)

Nm_Pr_F_Ig = Normalize(
    InFieldName = Pr_Fi_Ign, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition,
        ColorMap: RdYlBu_r
    ]
)

Nm_PcC_LgF = Normalize(
    InFieldName = Pc_Cg_LgFi, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Percent&nbsp;Change&nbsp;In&nbsp;Large&nbsp;Fire&nbsp;Probability,
        ColorMap: RdYlBu_r
    ]
)

# Operations

Fr_Rsk = WeightedSum(
    InFieldNames = [Nm_Lg_Fire, Nm_Pr_F_Ig, Nm_PcC_LgF], 
    Weights = [0.6, 0.2, 0.2],
    Metadata = [
        DisplayName: Fire&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Nm_Fr_Rsk = Normalize(
    InFieldName = Fr_Rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Fire&nbsp;Risk&nbsp;Index,
        ColorMap: RdYlBu_r
    ]
)


WUI_Rsk = Maximum(
    InFieldNames = [WUI_Interm, WUI_Interf],
    Metadata = [
        DisplayName: WUI&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)


Hi_Pri = WeightedSum(
    InFieldNames = [Nm_Fr_Rsk, Nm_Fl_Fa, LU_Cat_Cvt, WUI_Rsk], 
    Weights = [0.3, 0.1, 0.3, 0.3],
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Site&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Nm_Hi_Pri = Normalize(
    InFieldName = Hi_Pri, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Site,
        ColorMap: RdYlBu_r
    ]
)

# Output

Out = EEMSWrite(
  OutFileName = {output_path},
  OutFieldNames = [Join_ID, Nm_Hi_Pri, Hi_Pri, Nm_Fr_Rsk, Fr_Rsk, Nm_Lg_Fire, Nm_Pr_F_Ig, Nm_PcC_LgF, LU_Cat_Cvt, Nm_Fl_Fa, Fl_Fa, WUI_Rsk, WUI_Interm, WUI_Interf, Pr_Lg_Fr, Pr_Fi_Ign, Pc_Cg_LgFi, LU_Cat_Cvt, LU_Cat_Fz, Mj_LU_Cat]
)
    """
    return base_str

In [33]:
reporting_units_path = os.path.join(THIS_FOLDER, "out_data/eems/A_RU_8.csv")
output_path = os.path.join(THIS_FOLDER, "out_data/eems/EEMS_Output.csv")
base_str = get_base_str(reporting_units_path, output_path)
f = open("EEMS_CMD_FILE.mpt", "w")
f.write(base_str)
f.close()

**Run EEMS Command `mpilot eems-csv [command_file]`**

In [34]:
# dir_path = os.path.join(THIS_FOLDER, "out_data/final")
# shutil.rmtree(dir_path)
# os.mkdir(dir_path)

eems = pd.merge(pd.read_csv("out_data/eems/EEMS_Output.csv"), pd.read_csv("out_data/eems/A_RU_8.csv")[["Join_ID", "geometry"]], on='Join_ID', how='inner')
eems['geometry'] = eems['geometry'].apply(wkt.loads)
crs_proj = CRS("EPSG:2229")
gdf = gpd.GeoDataFrame(eems, crs=crs_proj, geometry = eems.geometry)
gdf.to_file(driver = 'ESRI Shapefile', filename= "out_data/final/Final_EEMS_Output7.shp")

In [16]:
arcpy.ResetEnvironments()
gdb_name = "EEMS.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path
path = os.path.join(THIS_FOLDER, "out_data/final/Final_EEMS_Output.shp")
FeatureClass = arcpy.CopyFeatures_management(path, "EEMS")
crs = arcpy.SpatialReference('NAD 1983 UTM Zone 11N')
FeatureClass = arcpy.Project_management("EEMS", "EEMS_rpj", crs)
arcpy.management.Delete("EEMS")

<Result 'true'>

In [7]:
f1 = open("EEMS_CMD_FILE.mpt", 'r')
f2 = open("EEMS_CMD_FILE_2.mpt", 'w')
for line in f1:
    f2.write(line.replace('"', ''))
f1.close()
f2.close()

In [8]:
arcpy.ClearWorkspaceCache_management()
del FeatureClass